# Pandas - Přidávání a odstraňování sloupců a řádků
## Manipulace se strukturou DataFrame

---

## Úvod do manipulace struktury

### Proč měnit strukturu DataFrame
- **Přidávání sloupců** - výpočty, kategorizace, transformace dat
- **Přidávání řádků** - nová pozorování, aktualizace dat
- **Odstraňování sloupců** - čištění dat, redukce dimenzí
- **Odstraňování řádků** - filtrování, odstranění outlierů
- **Extrakce dat** - získání informací z existujících sloupců

### Základní přehled metod

| Operace | Metody | Příklad |
|---------|--------|---------|
| **Přidat sloupec** | `df['novy'] = hodnoty`, `.assign()`, `.insert()` | `df['kategorie'] = 'A'` |
| **Přidat řádek** | `.loc[index] = hodnoty` | `df.loc[len(df)] = [1,2,3]` |
| **Odstranit sloupec** | `.drop()`, `del df['col']` | `df.drop('col', axis=1)` |
| **Odstranit řádek** | `.drop()`, booleanovské filtrování | `df.drop(0, axis=0)` |

### Dokumentace
- [Pandas DataFrame dokumentace](https://pandas.pydata.org/docs/reference/frame.html)

---

## Přidávání sloupců

### Základní přiřazení

In [1]:
import pandas as pd
import numpy as np

# Ukázkový DataFrame - meteorologické stanice
meteo_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava', 'plzen'],
    'srazky': [456.2, 398.7, 512.3, 467.9],
    'teplota': [9.8, 10.2, 8.9, 9.5],
    'nadmorska_vyska': [237, 399, 210, 310]
})

print("Původní DataFrame:")
print(meteo_df)

# Přidání nového sloupce - konstantní hodnota
meteo_df['zeme'] = 'Česká republika'
print(f"\nPo přidání konstantního sloupce:")
print(meteo_df)

# Přidání sloupce s různými hodnotami
meteo_df['region'] = ['jižní morava', 'střední čechy', 'slezsko', 'západní čechy']
print(f"\nPo přidání sloupce s různými hodnotami:")
print(meteo_df)

# Přidání sloupce s výpočtem
meteo_df['srazky_cm'] = meteo_df['srazky'] / 10
print(f"\nPo přidání vypočítaného sloupce:")
print(meteo_df)

Původní DataFrame:
   stanice  srazky  teplota  nadmorska_vyska
0     brno   456.2      9.8              237
1    praha   398.7     10.2              399
2  ostrava   512.3      8.9              210
3    plzen   467.9      9.5              310

Po přidání konstantního sloupce:
   stanice  srazky  teplota  nadmorska_vyska             zeme
0     brno   456.2      9.8              237  Česká republika
1    praha   398.7     10.2              399  Česká republika
2  ostrava   512.3      8.9              210  Česká republika
3    plzen   467.9      9.5              310  Česká republika

Po přidání sloupce s různými hodnotami:
   stanice  srazky  teplota  nadmorska_vyska             zeme         region
0     brno   456.2      9.8              237  Česká republika   jižní morava
1    praha   398.7     10.2              399  Česká republika  střední čechy
2  ostrava   512.3      8.9              210  Česká republika        slezsko
3    plzen   467.9      9.5              310  Česká republika  

### Vytváření sloupců z existujících dat

In [2]:
# Nový DataFrame pro demonstraci
data_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava', 'plzen'],
    'srazky': [456.2, 398.7, 512.3, 467.9],
    'teplota': [9.8, 10.2, 8.9, 9.5],
    'nadmorska_vyska': [237, 399, 210, 310]
})

# Výpočty z více sloupců
data_df['komfort_index'] = (data_df['teplota'] * 0.7) + (data_df['srazky'] / 100)
print("Komfortní index (teplota*0.7 + srážky/100):")
print(data_df[['stanice', 'teplota', 'srazky', 'komfort_index']])

# Binární sloupce (True/False)
data_df['je_horska'] = data_df['nadmorska_vyska'] > 300
data_df['je_tepla'] = data_df['teplota'] > data_df['teplota'].median()

print(f"\nBinární kategorie:")
print(data_df[['stanice', 'nadmorska_vyska', 'je_horska', 'teplota', 'je_tepla']])

Komfortní index (teplota*0.7 + srážky/100):
   stanice  teplota  srazky  komfort_index
0     brno      9.8   456.2         11.422
1    praha     10.2   398.7         11.127
2  ostrava      8.9   512.3         11.353
3    plzen      9.5   467.9         11.329

Binární kategorie:
   stanice  nadmorska_vyska  je_horska  teplota  je_tepla
0     brno              237      False      9.8      True
1    praha              399       True     10.2      True
2  ostrava              210      False      8.9     False
3    plzen              310       True      9.5     False


### Metoda assign() - funkční styl

In [3]:
# assign() vytváří nový DataFrame (nemění původní)
print("assign() - funkční styl přidávání sloupců:")

clean_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava', 'plzen'],
    'srazky': [456.2, 398.7, 512.3, 467.9],
    'teplota': [9.8, 10.2, 8.9, 9.5]
})

# Základní assign
assign_df = clean_df.assign(
    srazky_mm=clean_df['srazky'],  # Kopie existujícího sloupce
    srazky_cm=clean_df['srazky'] / 10,  # Výpočet
    zeme='Česká republika'  # Konstanta
)

print("assign() s jednoduchými výpočty:")
print(assign_df)

# assign() s lambda funkcemi
lambda_df = clean_df.assign(
    teplota_f=lambda x: x['teplota'] * 9/5 + 32,  # Fahrenheit
    vlhkost_kategorie=lambda x: x['srazky'].apply(
        lambda val: 'sucho' if val < 420 else 'mokro'
    ),
    kombinovaný_index=lambda x: x['teplota'] + x['srazky']/100
)

print(f"\nassign() s lambda funkcemi:")
print(lambda_df)

# Výhody assign() oproti základnímu přiřazení
print(f"\nVýhody assign():")
print("1. Funkční styl - nemění původní DataFrame")
print("2. Možnost řetězení operací")  
print("3. Lambda funkce pro složitější výpočty")
print("4. Čitelnější kód pro více sloupců najednou")

# Porovnání
print(f"\nPůvodní DataFrame zůstal nezměněn:")
print(clean_df)

assign() - funkční styl přidávání sloupců:
assign() s jednoduchými výpočty:
   stanice  srazky  teplota  srazky_mm  srazky_cm             zeme
0     brno   456.2      9.8      456.2      45.62  Česká republika
1    praha   398.7     10.2      398.7      39.87  Česká republika
2  ostrava   512.3      8.9      512.3      51.23  Česká republika
3    plzen   467.9      9.5      467.9      46.79  Česká republika

assign() s lambda funkcemi:
   stanice  srazky  teplota  teplota_f vlhkost_kategorie  kombinovaný_index
0     brno   456.2      9.8      49.64             mokro             14.362
1    praha   398.7     10.2      50.36             sucho             14.187
2  ostrava   512.3      8.9      48.02             mokro             14.023
3    plzen   467.9      9.5      49.10             mokro             14.179

Výhody assign():
1. Funkční styl - nemění původní DataFrame
2. Možnost řetězení operací
3. Lambda funkce pro složitější výpočty
4. Čitelnější kód pro více sloupců najednou

Původn

### Vkládání sloupců na konkrétní pozici

In [4]:
# insert() - vložení sloupce na konkrétní pozici
insert_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava', 'plzen'],
    'srazky': [456.2, 398.7, 512.3, 467.9],
    'teplota': [9.8, 10.2, 8.9, 9.5]
})

# Vložení na pozici 1 (za první sloupec)
insert_df.insert(1, 'id_stanice', range(1, len(insert_df) + 1))
print("Vložení sloupce 'id_stanice' na pozici 1:")
print(insert_df)

# Vložení na začátek - nový DataFrame
new_insert_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava', 'plzen'],
    'srazky': [456.2, 398.7, 512.3, 467.9],
    'teplota': [9.8, 10.2, 8.9, 9.5]
})

new_insert_df.insert(0, 'poradi', ['A', 'B', 'C', 'D'])
print(f"\nVložení sloupce 'poradi' na začátek:")
print(new_insert_df)

# Vložení na konec - nový DataFrame
end_insert_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava', 'plzen'],
    'srazky': [456.2, 398.7, 512.3, 467.9],
    'teplota': [9.8, 10.2, 8.9, 9.5]
})

end_insert_df.insert(len(end_insert_df.columns), 'poznamka', 'aktivní')
print(f"\nVložení sloupce 'poznamka' na konec:")
print(end_insert_df)

# Kontrola pořadí sloupců
print(f"\nPořadí sloupců: {end_insert_df.columns.tolist()}")

Vložení sloupce 'id_stanice' na pozici 1:
   stanice  id_stanice  srazky  teplota
0     brno           1   456.2      9.8
1    praha           2   398.7     10.2
2  ostrava           3   512.3      8.9
3    plzen           4   467.9      9.5

Vložení sloupce 'poradi' na začátek:
  poradi  stanice  srazky  teplota
0      A     brno   456.2      9.8
1      B    praha   398.7     10.2
2      C  ostrava   512.3      8.9
3      D    plzen   467.9      9.5

Vložení sloupce 'poznamka' na konec:
   stanice  srazky  teplota poznamka
0     brno   456.2      9.8  aktivní
1    praha   398.7     10.2  aktivní
2  ostrava   512.3      8.9  aktivní
3    plzen   467.9      9.5  aktivní

Pořadí sloupců: ['stanice', 'srazky', 'teplota', 'poznamka']


---

## Podmíněné přiřazení pomocí loc

### Teorie loc přiřazení
**Metoda `.loc[]`** umožňuje **podmíněné přiřazení hodnot** na základě booleanovských podmínek. Je to efektivní způsob, jak vytvářet kategorizované sloupce nebo měnit hodnoty podle kritérií.

**Základní syntaxe:**
- `df.loc[podmínka, sloupec] = hodnota`
- `df.loc[podmínka, [sloupce]] = hodnoty`

**Výhody loc přiřazení:**
- **Výkonnost** - rychlejší než cykly
- **Čitelnost** - jasně viditelné podmínky
- **Flexibilita** - složité podmínky s `&`, `|`, `~`
- **Vektorizace** - operace nad celými sloupci

### Základní podmíněné přiřazení

In [5]:
# DataFrame pro demonstraci loc přiřazení
loc_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava', 'plzen'],
    'srazky': [456.2, 398.7, 512.3, 467.9],
    'teplota': [9.8, 10.2, 8.9, 9.5],
    'nadmorska_vyska': [237, 399, 210, 310]
})

print("Původní DataFrame:")
print(loc_df)

# Kategorizace podle podmínek - nejprve výchozí hodnota
loc_df['kategorie_srazky'] = 'normální'
loc_df.loc[loc_df['srazky'] > 450, 'kategorie_srazky'] = 'vysoké'
loc_df.loc[loc_df['srazky'] < 400, 'kategorie_srazky'] = 'nízké'

print(f"\nKategorizace podle srážek:")
print(loc_df[['stanice', 'srazky', 'kategorie_srazky']])

Původní DataFrame:
   stanice  srazky  teplota  nadmorska_vyska
0     brno   456.2      9.8              237
1    praha   398.7     10.2              399
2  ostrava   512.3      8.9              210
3    plzen   467.9      9.5              310

Kategorizace podle srážek:
   stanice  srazky kategorie_srazky
0     brno   456.2           vysoké
1    praha   398.7            nízké
2  ostrava   512.3           vysoké
3    plzen   467.9           vysoké


### Pokročilé podmíněné přiřazení

In [6]:
# Nový DataFrame pro složitější příklady
complex_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava', 'plzen', 'liberec'],
    'srazky': [456.2, 398.7, 512.3, 467.9, 423.8],
    'teplota': [9.8, 10.2, 8.9, 9.5, 8.1],
    'nadmorska_vyska': [237, 399, 210, 310, 376]
})

# Kombinace podmínek s & (AND) a | (OR)
complex_df['kategorie_lokality'] = 'neurceno'

# Horská a studená místa
complex_df.loc[
    (complex_df['nadmorska_vyska'] > 300) & (complex_df['teplota'] < 9), 
    'kategorie_lokality'
] = 'horská_studená'

# Nížinná a teplá místa
complex_df.loc[
    (complex_df['nadmorska_vyska'] <= 300) & (complex_df['teplota'] >= 9.5), 
    'kategorie_lokality'
] = 'nížinná_teplá'

# Místa s vysokými srážkami NEBO vysokou nadmořskou výškou
complex_df.loc[
    (complex_df['srazky'] > 500) | (complex_df['nadmorska_vyska'] > 350), 
    'kategorie_lokality'
] = 'extrémní'

print("Složité kategorizace:")
print(complex_df[['stanice', 'nadmorska_vyska', 'teplota', 'srazky', 'kategorie_lokality']])

# Přiřazení více sloupců najednou
complex_df.loc[complex_df['teplota'] > 9, ['je_tepla', 'doporuceni']] = [True, 'vhodná_lokalita']
complex_df.loc[complex_df['teplota'] <= 9, ['je_tepla', 'doporuceni']] = [False, 'chladná_lokalita']

print(f"\nPřiřazení více sloupců:")
print(complex_df[['stanice', 'teplota', 'je_tepla', 'doporuceni']])

Složité kategorizace:
   stanice  nadmorska_vyska  teplota  srazky kategorie_lokality
0     brno              237      9.8   456.2      nížinná_teplá
1    praha              399     10.2   398.7           extrémní
2  ostrava              210      8.9   512.3           extrémní
3    plzen              310      9.5   467.9           neurceno
4  liberec              376      8.1   423.8           extrémní

Přiřazení více sloupců:
   stanice  teplota je_tepla        doporuceni
0     brno      9.8     True   vhodná_lokalita
1    praha     10.2     True   vhodná_lokalita
2  ostrava      8.9    False  chladná_lokalita
3    plzen      9.5     True   vhodná_lokalita
4  liberec      8.1    False  chladná_lokalita


C:\Users\petrrozkosny\AppData\Local\Temp\ipykernel_3648\2996938334.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  complex_df.loc[complex_df['teplota'] > 9, ['je_tepla', 'doporuceni']] = [True, 'vhodná_lokalita']
C:\Users\petrrozkosny\AppData\Local\Temp\ipykernel_3648\2996938334.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'vhodná_lokalita' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  complex_df.loc[complex_df['teplota'] > 9, ['je_tepla', 'doporuceni']] = [True, 'vhodná_lokalita']


### Použití negace (~) a isin()

In [7]:
# DataFrame pro demonstraci pokročilých podmínek
advanced_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava', 'plzen', 'liberec', 'jihlava'],
    'region': ['morava', 'čechy', 'slezsko', 'čechy', 'čechy', 'čechy'],
    'srazky': [456.2, 398.7, 512.3, 467.9, 423.8, 445.7],
    'teplota': [9.8, 10.2, 8.9, 9.5, 8.1, 9.3]
})

# Použití negace (~) - všechno kromě Čech
advanced_df['je_cechy'] = True
advanced_df.loc[~(advanced_df['region'] == 'čechy'), 'je_cechy'] = False

print("Použití negace (~):")
print(advanced_df[['stanice', 'region', 'je_cechy']])

# Použití isin() pro výběr z více možností
ceske_mesta = ['praha', 'plzen', 'liberec', 'jihlava']
advanced_df['je_ceske_mesto'] = False
advanced_df.loc[advanced_df['stanice'].isin(ceske_mesta), 'je_ceske_mesto'] = True

print(f"\nPoužití isin():")
print(advanced_df[['stanice', 'je_ceske_mesto']])

# Kombinace isin() s dalšími podmínkami
advanced_df['priority'] = 'normální'
advanced_df.loc[
    advanced_df['stanice'].isin(['praha', 'brno']) & (advanced_df['teplota'] > 9.5), 
    'priority'
] = 'vysoká'

print(f"\nKombinace isin() s podmínkami:")
print(advanced_df[['stanice', 'teplota', 'priority']])

Použití negace (~):
   stanice   region  je_cechy
0     brno   morava     False
1    praha    čechy      True
2  ostrava  slezsko     False
3    plzen    čechy      True
4  liberec    čechy      True
5  jihlava    čechy      True

Použití isin():
   stanice  je_ceske_mesto
0     brno           False
1    praha            True
2  ostrava           False
3    plzen            True
4  liberec            True
5  jihlava            True

Kombinace isin() s podmínkami:
   stanice  teplota  priority
0     brno      9.8    vysoká
1    praha     10.2    vysoká
2  ostrava      8.9  normální
3    plzen      9.5  normální
4  liberec      8.1  normální
5  jihlava      9.3  normální


---

## Metoda apply() pro transformace

### Teorie metody apply()
**Metoda `.apply()`** umožňuje aplikovat **vlastní funkce** na sloupce nebo řádky DataFrame. Je to klíčový nástroj pro **složitější transformace**, které nelze řešit jednoduchými operátory.

**Základní typy apply():**
- **`Series.apply(funkce)`** - aplikuje funkci na každý prvek sloupce
- **`DataFrame.apply(funkce, axis=0)`** - aplikuje funkci na sloupce
- **`DataFrame.apply(funkce, axis=1)`** - aplikuje funkci na řádky

**Kdy použít apply():**
- **Složité výpočty** vyžadující vlastní logiku
- **Kategorizace** podle složitých pravidel
- **Textové transformace** vyžadující více kroků
- **Podmíněná logika** s více větvemi

### Základní použití apply() na sloupce

In [8]:
# DataFrame pro demonstraci apply()
apply_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava', 'plzen'],
    'srazky_mm': [456.2, 398.7, 512.3, 467.9],
    'teplota_c': [9.8, 10.2, 8.9, 9.5],
    'tlak_hpa': [1013.2, 1011.8, 1015.1, 1012.5]
})

print("Původní DataFrame:")
print(apply_df)

# Jednoduchá funkce pro kategorizaci srážek
def kategorizuj_srazky(srazky):
    if srazky < 400:
        return 'nízké'
    elif srazky < 500:
        return 'střední'
    else:
        return 'vysoké'

# Aplikace funkce na sloupec
apply_df['kategorie_srazky'] = apply_df['srazky_mm'].apply(kategorizuj_srazky)

print(f"\nKategorizace srážek pomocí apply():")
print(apply_df[['stanice', 'srazky_mm', 'kategorie_srazky']])

Původní DataFrame:
   stanice  srazky_mm  teplota_c  tlak_hpa
0     brno      456.2        9.8    1013.2
1    praha      398.7       10.2    1011.8
2  ostrava      512.3        8.9    1015.1
3    plzen      467.9        9.5    1012.5

Kategorizace srážek pomocí apply():
   stanice  srazky_mm kategorie_srazky
0     brno      456.2          střední
1    praha      398.7            nízké
2  ostrava      512.3           vysoké
3    plzen      467.9          střední


### Jednoduché funkce s apply()

In [9]:
# Jednoduchá funkce pro kategorizaci teploty
def je_teplo(teplota):
    """Jednoduchá kategorizace teploty"""
    if teplota >= 10:
        return 'teplo'
    else:
        return 'chladno'

# Aplikace jednoduché funkce
simple_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava'],
    'teplota_c': [9.8, 10.2, 8.9]
})

simple_df['teplo_chladno'] = simple_df['teplota_c'].apply(je_teplo)

print("Jednoduchá kategorizace:")
print(simple_df)

Jednoduchá kategorizace:
   stanice  teplota_c teplo_chladno
0     brno        9.8       chladno
1    praha       10.2         teplo
2  ostrava        8.9       chladno


### Lambda funkce s apply()

In [10]:
# Lambda funkce pro rychlé transformace
lambda_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava'],
    'teplota_c': [9.8, 10.2, 8.9],
    'srazky_mm': [456.2, 398.7, 512.3]
})

# Lambda pro jednoduché výpočty
lambda_df['teplota_f'] = lambda_df['teplota_c'].apply(lambda x: x * 9/5 + 32)

# Lambda pro podmíněnou logiku
lambda_df['doporuceni'] = lambda_df['srazky_mm'].apply(
    lambda x: 'vezmi_deštník' if x > 450 else 'není_potřeba'
)

# Lambda pro textové transformace
lambda_df['stanice_velka'] = lambda_df['stanice'].apply(lambda x: x.upper())

print("Lambda funkce s apply():")
print(lambda_df)

Lambda funkce s apply():
   stanice  teplota_c  srazky_mm  teplota_f     doporuceni stanice_velka
0     brno        9.8      456.2      49.64  vezmi_deštník          BRNO
1    praha       10.2      398.7      50.36   není_potřeba         PRAHA
2  ostrava        8.9      512.3      48.02  vezmi_deštník       OSTRAVA


### Apply() na řádky (axis=1)

In [11]:
# Apply na řádky pro výpočty z více sloupců
row_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava'],
    'teplota_c': [9.8, 10.2, 8.9],
    'srazky_mm': [456.2, 398.7, 512.3]
})

def secti_hodnoty(radek):
    """Jednoduché sečtení teploty a srážek"""
    return radek['teplota_c'] + radek['srazky_mm']

# Aplikace funkce na řádky
row_df['soucet'] = row_df.apply(secti_hodnoty, axis=1)

print("Apply() na řádky (axis=1):")
print(row_df)

# Lambda na řádky pro jednodušší výpočty
row_df['rozdil'] = row_df.apply(
    lambda row: row['srazky_mm'] - row['teplota_c'], axis=1
)

print(f"\nLambda na řádky:")
print(row_df[['stanice', 'rozdil']])

Apply() na řádky (axis=1):
   stanice  teplota_c  srazky_mm  soucet
0     brno        9.8      456.2   466.0
1    praha       10.2      398.7   408.9
2  ostrava        8.9      512.3   521.2

Lambda na řádky:
   stanice  rozdil
0     brno   446.4
1    praha   388.5
2  ostrava   503.4


---

## Přidávání řádků

### Přidání jednoho řádku pomocí loc

In [12]:
# Přidání nového řádku na konec
row_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava'],
    'srazky': [456.2, 398.7, 512.3],
    'teplota': [9.8, 10.2, 8.9]
})

print("Původní DataFrame:")
print(row_df)

# Nový řádek na pozici len(df)
row_df.loc[len(row_df)] = ['liberec', 423.8, 8.1]
print(f"\nPo přidání řádku:")
print(row_df)

# Nový DataFrame pro pojmenovaný index
named_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava'],
    'srazky': [456.2, 398.7, 512.3],
    'teplota': [9.8, 10.2, 8.9]
})

named_df.loc['novy_radek'] = ['ceske_budejovice', 445.3, 9.2]
print(f"\nPo přidání pojmenovaného řádku:")
print(named_df)

# Nový DataFrame pro neúplný řádek
partial_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava'],
    'srazky': [456.2, 398.7, 512.3],
    'teplota': [9.8, 10.2, 8.9]
})

partial_df.loc['neuplny'] = ['pardubice', np.nan, 9.7]
print(f"\nPo přidání neúplného řádku:")
print(partial_df)

Původní DataFrame:
   stanice  srazky  teplota
0     brno   456.2      9.8
1    praha   398.7     10.2
2  ostrava   512.3      8.9

Po přidání řádku:
   stanice  srazky  teplota
0     brno   456.2      9.8
1    praha   398.7     10.2
2  ostrava   512.3      8.9
3  liberec   423.8      8.1

Po přidání pojmenovaného řádku:
                     stanice  srazky  teplota
0                       brno   456.2      9.8
1                      praha   398.7     10.2
2                    ostrava   512.3      8.9
novy_radek  ceske_budejovice   445.3      9.2

Po přidání neúplného řádku:
           stanice  srazky  teplota
0             brno   456.2      9.8
1            praha   398.7     10.2
2          ostrava   512.3      8.9
neuplny  pardubice     NaN      9.7


---

## Odstraňování sloupců

### Základní metody odstranění sloupců

In [13]:
# DataFrame s extra sloupci
extra_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava', 'plzen'],
    'srazky': [456.2, 398.7, 512.3, 467.9],
    'teplota': [9.8, 10.2, 8.9, 9.5],
    'extra1': ['hodnota1', 'hodnota1', 'hodnota1', 'hodnota1'],
    'extra2': ['hodnota2', 'hodnota2', 'hodnota2', 'hodnota2'],
    'extra3': ['hodnota3', 'hodnota3', 'hodnota3', 'hodnota3']
})

print("DataFrame s extra sloupci:")
print(extra_df)

# Metoda 1: drop() - vytváří kopii
bez_extra1 = extra_df.drop('extra1', axis=1)
print(f"\nPo odstranění 'extra1' pomocí drop():")
print(bez_extra1)
print(f"Původní DataFrame zůstal nezměněn: {extra_df.columns.tolist()}")

# Metoda 2: drop() inplace - mění původní
inplace_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava', 'plzen'],
    'srazky': [456.2, 398.7, 512.3, 467.9],
    'teplota': [9.8, 10.2, 8.9, 9.5],
    'extra1': ['hodnota1', 'hodnota1', 'hodnota1', 'hodnota1'],
    'extra2': ['hodnota2', 'hodnota2', 'hodnota2', 'hodnota2']
})

inplace_df.drop('extra2', axis=1, inplace=True)
print(f"\nPo inplace drop('extra2'):")
print(inplace_df)

# Metoda 3: del - odstraní přímo z původního
del_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava', 'plzen'],
    'srazky': [456.2, 398.7, 512.3, 467.9],
    'teplota': [9.8, 10.2, 8.9, 9.5],
    'extra3': ['hodnota3', 'hodnota3', 'hodnota3', 'hodnota3']
})

del del_df['extra3']
print(f"\nPo del statement:")
print(del_df)

DataFrame s extra sloupci:
   stanice  srazky  teplota    extra1    extra2    extra3
0     brno   456.2      9.8  hodnota1  hodnota2  hodnota3
1    praha   398.7     10.2  hodnota1  hodnota2  hodnota3
2  ostrava   512.3      8.9  hodnota1  hodnota2  hodnota3
3    plzen   467.9      9.5  hodnota1  hodnota2  hodnota3

Po odstranění 'extra1' pomocí drop():
   stanice  srazky  teplota    extra2    extra3
0     brno   456.2      9.8  hodnota2  hodnota3
1    praha   398.7     10.2  hodnota2  hodnota3
2  ostrava   512.3      8.9  hodnota2  hodnota3
3    plzen   467.9      9.5  hodnota2  hodnota3
Původní DataFrame zůstal nezměněn: ['stanice', 'srazky', 'teplota', 'extra1', 'extra2', 'extra3']

Po inplace drop('extra2'):
   stanice  srazky  teplota    extra1
0     brno   456.2      9.8  hodnota1
1    praha   398.7     10.2  hodnota1
2  ostrava   512.3      8.9  hodnota1
3    plzen   467.9      9.5  hodnota1

Po del statement:
   stanice  srazky  teplota
0     brno   456.2      9.8
1    praha   

### Odstranění více sloupců najednou

In [14]:
# Nový DataFrame pro množné odstranění
multi_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava', 'plzen'],
    'srazky': [456.2, 398.7, 512.3, 467.9],
    'teplota': [9.8, 10.2, 8.9, 9.5],
    'extra1': ['hodnota1', 'hodnota1', 'hodnota1', 'hodnota1'],
    'extra2': ['hodnota2', 'hodnota2', 'hodnota2', 'hodnota2'],
    'extra3': ['hodnota3', 'hodnota3', 'hodnota3', 'hodnota3']
})

# Odstranění více sloupců seznamem
bez_extra_sloupcu = multi_df.drop(['extra1', 'extra2', 'extra3'], axis=1)
print("Odstranění více sloupců najednou:")
print(bez_extra_sloupcu)

# Odstranění pomocí podmínek
# Najít sloupce začínající na 'extra'
extra_sloupce = [col for col in multi_df.columns if col.startswith('extra')]
print(f"\nExtra sloupce: {extra_sloupce}")

bez_extra_podminka = multi_df.drop(columns=extra_sloupce)
print("Odstranění podle podmínky:")
print(bez_extra_podminka)

# Ponechání pouze vybraných sloupců
pouze_zakladni = multi_df[['stanice', 'srazky', 'teplota']]
print(f"\nPonechání pouze základních sloupců:")
print(pouze_zakladni)

Odstranění více sloupců najednou:
   stanice  srazky  teplota
0     brno   456.2      9.8
1    praha   398.7     10.2
2  ostrava   512.3      8.9
3    plzen   467.9      9.5

Extra sloupce: ['extra1', 'extra2', 'extra3']
Odstranění podle podmínky:
   stanice  srazky  teplota
0     brno   456.2      9.8
1    praha   398.7     10.2
2  ostrava   512.3      8.9
3    plzen   467.9      9.5

Ponechání pouze základních sloupců:
   stanice  srazky  teplota
0     brno   456.2      9.8
1    praha   398.7     10.2
2  ostrava   512.3      8.9
3    plzen   467.9      9.5


---

## Odstraňování řádků

### Odstranění podle indexu

In [15]:
# DataFrame s více řádky pro testování
rows_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava', 'plzen', 'liberec', 'olomouc'],
    'srazky': [456.2, 398.7, 512.3, 467.9, 423.8, 456.8],
    'teplota': [9.8, 10.2, 8.9, 9.5, 8.1, 8.7]
})

print("DataFrame pro testování odstranění řádků:")
print(rows_df)

# Odstranění jednoho řádku podle indexu
bez_prvniho = rows_df.drop(0, axis=0)
print(f"\nBez prvního řádku (index 0):")
print(bez_prvniho)

# Odstranění více řádků
bez_prvnich_dvou = rows_df.drop([0, 1], axis=0)
print(f"\nBez prvních dvou řádků:")
print(bez_prvnich_dvou)

# Odstranění podle rozsahu (slicing)
prostredni_cast = rows_df.drop(range(1, 4), axis=0)
print(f"\nBez řádků 1-3:")
print(prostredni_cast)

DataFrame pro testování odstranění řádků:
   stanice  srazky  teplota
0     brno   456.2      9.8
1    praha   398.7     10.2
2  ostrava   512.3      8.9
3    plzen   467.9      9.5
4  liberec   423.8      8.1
5  olomouc   456.8      8.7

Bez prvního řádku (index 0):
   stanice  srazky  teplota
1    praha   398.7     10.2
2  ostrava   512.3      8.9
3    plzen   467.9      9.5
4  liberec   423.8      8.1
5  olomouc   456.8      8.7

Bez prvních dvou řádků:
   stanice  srazky  teplota
2  ostrava   512.3      8.9
3    plzen   467.9      9.5
4  liberec   423.8      8.1
5  olomouc   456.8      8.7

Bez řádků 1-3:
   stanice  srazky  teplota
0     brno   456.2      9.8
4  liberec   423.8      8.1
5  olomouc   456.8      8.7


### Odstranění podle podmínek (booleanovské filtrování)

In [16]:
# Filtrování jako alternativa k drop()
filter_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava', 'plzen', 'liberec', 'olomouc'],
    'srazky': [456.2, 398.7, 512.3, 467.9, 423.8, 456.8],
    'teplota': [9.8, 10.2, 8.9, 9.5, 8.1, 8.7]
})

print("Původní DataFrame:")
print(filter_df)

# Ponechání pouze řádků splňujících podmínku
vysoké_srazky = filter_df.loc[filter_df['srazky'] > 450]
print("\nPouze stanice s vysokými srážkami (>450mm):")
print(vysoké_srazky)

# Odstranění řádků nesplňujících podmínku (ekvivalent k drop())
bez_nizkych_srazek = filter_df.loc[~(filter_df['srazky'] < 400)]
print(f"\nBez stanic s nízkými srážkami (<400mm):")
print(bez_nizkych_srazek)

# Kombinace podmínek
filtrovane = filter_df.loc[
    (filter_df['srazky'] >= 400) & 
    (filter_df['teplota'] >= 8.5)
]
print(f"\nStanice s přiměřenými srážkami (≥400mm) a teplotou (≥8.5°C):")
print(filtrovane)

Původní DataFrame:
   stanice  srazky  teplota
0     brno   456.2      9.8
1    praha   398.7     10.2
2  ostrava   512.3      8.9
3    plzen   467.9      9.5
4  liberec   423.8      8.1
5  olomouc   456.8      8.7

Pouze stanice s vysokými srážkami (>450mm):
   stanice  srazky  teplota
0     brno   456.2      9.8
2  ostrava   512.3      8.9
3    plzen   467.9      9.5
5  olomouc   456.8      8.7

Bez stanic s nízkými srážkami (<400mm):
   stanice  srazky  teplota
0     brno   456.2      9.8
2  ostrava   512.3      8.9
3    plzen   467.9      9.5
4  liberec   423.8      8.1
5  olomouc   456.8      8.7

Stanice s přiměřenými srážkami (≥400mm) a teplotou (≥8.5°C):
   stanice  srazky  teplota
0     brno   456.2      9.8
2  ostrava   512.3      8.9
3    plzen   467.9      9.5
5  olomouc   456.8      8.7


### Odstranění duplicitních řádků

In [17]:
# Vytvoření dat s duplicity
data_s_duplikaty = pd.DataFrame({
    'stanice': ['brno', 'praha', 'brno', 'ostrava', 'praha'],
    'srazky': [456.2, 398.7, 456.2, 512.3, 398.7],
    'teplota': [9.8, 10.2, 9.8, 8.9, 10.2]
})

print("Data s duplicitními řádky:")
print(data_s_duplikaty)

# Odstranění duplicitních řádků
bez_duplikatu = data_s_duplikaty.drop_duplicates()
print(f"\nBez úplných duplicitů:")
print(bez_duplikatu)

# Odstranění duplicit podle konkrétních sloupců
bez_duplikatu_stanice = data_s_duplikaty.drop_duplicates(subset=['stanice'])
print(f"\nBez duplicit podle sloupce 'stanice':")
print(bez_duplikatu_stanice)

# Zachování posledního výskytu místo prvního
posledni_duplikaty = data_s_duplikaty.drop_duplicates(subset=['stanice'], keep='last')
print(f"\nZachování posledního výskytu:")
print(posledni_duplikaty)

Data s duplicitními řádky:
   stanice  srazky  teplota
0     brno   456.2      9.8
1    praha   398.7     10.2
2     brno   456.2      9.8
3  ostrava   512.3      8.9
4    praha   398.7     10.2

Bez úplných duplicitů:
   stanice  srazky  teplota
0     brno   456.2      9.8
1    praha   398.7     10.2
3  ostrava   512.3      8.9

Bez duplicit podle sloupce 'stanice':
   stanice  srazky  teplota
0     brno   456.2      9.8
1    praha   398.7     10.2
3  ostrava   512.3      8.9

Zachování posledního výskytu:
   stanice  srazky  teplota
2     brno   456.2      9.8
3  ostrava   512.3      8.9
4    praha   398.7     10.2


### Odstranění řádků s chybějícími hodnotami

In [18]:
# Vytvoření dat s NaN
data_s_nan = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava', 'plzen'],
    'srazky': [456.2, np.nan, 512.3, 467.9],
    'teplota': [9.8, 10.2, np.nan, 9.5],
    'tlak': [1013.2, 1011.8, 1015.1, np.nan]
})

print("Data s NaN hodnotami:")
print(data_s_nan)

# Odstranění řádků s jakýmikoli NaN
bez_nan_any = data_s_nan.dropna()
print(f"\nBez řádků s jakýmikoli NaN:")
print(bez_nan_any)

# Odstranění pouze pokud jsou všechny hodnoty NaN
bez_nan_all = data_s_nan.dropna(how='all')
print(f"\nBez řádků kde jsou všechny hodnoty NaN:")
print(bez_nan_all)

# Odstranění podle konkrétních sloupců
bez_nan_srazky = data_s_nan.dropna(subset=['srazky'])
print(f"\nBez řádků s NaN ve sloupci 'srazky':")
print(bez_nan_srazky)

# Prahová hodnota - minimální počet platných hodnot
min_platnych = data_s_nan.dropna(thresh=3)  # Alespoň 3 platné hodnoty
print(f"\nŘádky s alespoň 3 platnými hodnotami:")
print(min_platnych)

Data s NaN hodnotami:
   stanice  srazky  teplota    tlak
0     brno   456.2      9.8  1013.2
1    praha     NaN     10.2  1011.8
2  ostrava   512.3      NaN  1015.1
3    plzen   467.9      9.5     NaN

Bez řádků s jakýmikoli NaN:
  stanice  srazky  teplota    tlak
0    brno   456.2      9.8  1013.2

Bez řádků kde jsou všechny hodnoty NaN:
   stanice  srazky  teplota    tlak
0     brno   456.2      9.8  1013.2
1    praha     NaN     10.2  1011.8
2  ostrava   512.3      NaN  1015.1
3    plzen   467.9      9.5     NaN

Bez řádků s NaN ve sloupci 'srazky':
   stanice  srazky  teplota    tlak
0     brno   456.2      9.8  1013.2
2  ostrava   512.3      NaN  1015.1
3    plzen   467.9      9.5     NaN

Řádky s alespoň 3 platnými hodnotami:
   stanice  srazky  teplota    tlak
0     brno   456.2      9.8  1013.2
1    praha     NaN     10.2  1011.8
2  ostrava   512.3      NaN  1015.1
3    plzen   467.9      9.5     NaN


---

## Extrakce dat z existujících sloupců

### Práce s textovými sloupci - str accessor

In [19]:
# DataFrame s textovými daty
text_df = pd.DataFrame({
    'nazev_stanice': ['Brno-Tuřany', 'Praha-Ruzyně', 'Ostrava-Mošnov', 'Plzeň-Líně'],
    'kód_stanice': ['CZ001', 'CZ002', 'CZ003', 'CZ004'],
    'popis': ['Jihomoravský kraj, letiště', 'Středočeský kraj, mezinárodní letiště', 
              'Moravskoslezský kraj, letiště', 'Plzeňský kraj, regionální letiště']
})

print("DataFrame s textovými daty:")
print(text_df)

# Základní textové operace
text_df['mesto'] = text_df['nazev_stanice'].str.split('-').str[0]
text_df['mesto_lower'] = text_df['mesto'].str.lower()
text_df['mesto_upper'] = text_df['mesto'].str.upper()

print(f"\nExtrakce názvu města:")
print(text_df[['nazev_stanice', 'mesto', 'mesto_lower', 'mesto_upper']])

# Pokročilejší textové operace
text_df['delka_nazvu'] = text_df['nazev_stanice'].str.len()
text_df['obsahuje_letiste'] = text_df['popis'].str.contains('letiště')
text_df['cislo_kodu'] = text_df['kód_stanice'].str.extract(r'(\d+)')

print(f"\nPokročilé textové extrakce:")
print(text_df[['nazev_stanice', 'delka_nazvu', 'obsahuje_letiste', 'cislo_kodu']])

DataFrame s textovými daty:
    nazev_stanice kód_stanice                                  popis
0     Brno-Tuřany       CZ001             Jihomoravský kraj, letiště
1    Praha-Ruzyně       CZ002  Středočeský kraj, mezinárodní letiště
2  Ostrava-Mošnov       CZ003          Moravskoslezský kraj, letiště
3      Plzeň-Líně       CZ004      Plzeňský kraj, regionální letiště

Extrakce názvu města:
    nazev_stanice    mesto mesto_lower mesto_upper
0     Brno-Tuřany     Brno        brno        BRNO
1    Praha-Ruzyně    Praha       praha       PRAHA
2  Ostrava-Mošnov  Ostrava     ostrava     OSTRAVA
3      Plzeň-Líně    Plzeň       plzeň       PLZEŇ

Pokročilé textové extrakce:
    nazev_stanice  delka_nazvu  obsahuje_letiste cislo_kodu
0     Brno-Tuřany           11              True        001
1    Praha-Ruzyně           12              True        002
2  Ostrava-Mošnov           14              True        003
3      Plzeň-Líně           10              True        004


### Práce s číselnými sloupci

In [20]:
# Matematické operace a transformace
numeric_df = pd.DataFrame({
    'stanice': ['brno', 'praha', 'ostrava'],
    'teplota_c': [9.8, 10.2, 8.9],
    'srazky_mm': [456.2, 398.7, 512.3],
    'tlak_hpa': [1013.2, 1011.8, 1015.1]
})

print("Číselná data pro transformace:")
print(numeric_df)

# Konverze jednotek
numeric_df['teplota_f'] = numeric_df['teplota_c'] * 9/5 + 32
numeric_df['srazky_cm'] = numeric_df['srazky_mm'] / 10
numeric_df['tlak_mmhg'] = numeric_df['tlak_hpa'] * 0.750062

print(f"\nKonverze jednotek:")
print(numeric_df[['stanice', 'teplota_c', 'teplota_f', 'srazky_mm', 'srazky_cm']])

# Matematické funkce
numeric_df['log_srazky'] = np.log(numeric_df['srazky_mm'])
numeric_df['sqrt_tlak'] = np.sqrt(numeric_df['tlak_hpa'])
numeric_df['teplota_rounded'] = numeric_df['teplota_c'].round(0)

print(f"\nMatematické transformace:")
print(numeric_df[['stanice', 'srazky_mm', 'log_srazky', 'teplota_c', 'teplota_rounded']])

Číselná data pro transformace:
   stanice  teplota_c  srazky_mm  tlak_hpa
0     brno        9.8      456.2    1013.2
1    praha       10.2      398.7    1011.8
2  ostrava        8.9      512.3    1015.1

Konverze jednotek:
   stanice  teplota_c  teplota_f  srazky_mm  srazky_cm
0     brno        9.8      49.64      456.2      45.62
1    praha       10.2      50.36      398.7      39.87
2  ostrava        8.9      48.02      512.3      51.23

Matematické transformace:
   stanice  srazky_mm  log_srazky  teplota_c  teplota_rounded
0     brno      456.2    6.122931        9.8             10.0
1    praha      398.7    5.988209       10.2             10.0
2  ostrava      512.3    6.238910        8.9              9.0


### Práce s datetime sloupci - dt accessor

In [21]:
# Časová data
time_df = pd.DataFrame({
    'datum': pd.date_range('2023-01-01', periods=6, freq='M'),
    'teplota': [2.1, 5.8, 12.3, 18.9, 23.4, 26.7],
    'srazky': [45.2, 38.7, 52.1, 61.5, 78.3, 89.4]
})

print("Časová data:")
print(time_df)

# Extrakce komponent z datetime
time_df['rok'] = time_df['datum'].dt.year
time_df['mesic'] = time_df['datum'].dt.month
time_df['nazev_mesice'] = time_df['datum'].dt.month_name()
time_df['ctvrtleti'] = time_df['datum'].dt.quarter
time_df['den_v_roce'] = time_df['datum'].dt.dayofyear

print(f"\nExtrakce z datetime:")
print(time_df[['datum', 'mesic', 'nazev_mesice', 'ctvrtleti']])

# Časové výpočty
time_df['dny_od_zacatku'] = (time_df['datum'] - time_df['datum'].min()).dt.days
time_df['je_leto'] = time_df['mesic'].isin([6, 7, 8])

print(f"\nČasové výpočty:")
print(time_df[['datum', 'dny_od_zacatku', 'je_leto']])

Časová data:
       datum  teplota  srazky
0 2023-01-31      2.1    45.2
1 2023-02-28      5.8    38.7
2 2023-03-31     12.3    52.1
3 2023-04-30     18.9    61.5
4 2023-05-31     23.4    78.3
5 2023-06-30     26.7    89.4

Extrakce z datetime:
       datum  mesic nazev_mesice  ctvrtleti
0 2023-01-31      1      January          1
1 2023-02-28      2     February          1
2 2023-03-31      3        March          1
3 2023-04-30      4        April          2
4 2023-05-31      5          May          2
5 2023-06-30      6         June          2

Časové výpočty:
       datum  dny_od_zacatku  je_leto
0 2023-01-31               0    False
1 2023-02-28              28    False
2 2023-03-31              59    False
3 2023-04-30              89    False
4 2023-05-31             120    False
5 2023-06-30             150     True
